**Secrets**

The secrets below  like the Cosmos account key are retrieved from a secret scope. If you don't have defined a secret scope for a Cosmos Account you want to use when going through this sample you can find the instructions on how to create one here:
- Here you can [Create a new secret scope](./#secrets/createScope) for the current Databricks workspace
  - See how you can create an [Azure Key Vault backed secret scope](https://docs.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes#--create-an-azure-key-vault-backed-secret-scope) 
  - See how you can create a [Databricks backed secret scope](https://docs.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes#create-a-databricks-backed-secret-scope)
- And here you can find information on how to [add secrets to your Spark configuration](https://docs.microsoft.com/en-us/azure/databricks/security/secrets/secrets#read-a-secret)
If you don't want to use secrets at all you can of course also just assign the values in clear-text below - but for obvious reasons we recommend the usage of secrets.

In [0]:
cosmosEndpoint = spark.conf.get("spark.cosmos.accountEndpoint")
cosmosMasterKey = spark.conf.get("spark.cosmos.accountKey")

**Preparation - loading data source "[NYC Taxi & Limousine Commission - yellow taxi trip records](https://azure.microsoft.com/en-us/services/open-datasets/catalog/nyc-taxi-limousine-commission-yellow-taxi-trip-records/)"**

The yellow taxi trip records include fields capturing pick-up and drop-off dates/times, pick-up and drop-off locations, trip distances, itemized fares, rate types, payment types, and driver-reported passenger counts. This data set has over 1.5 billion records (>50 GB) of data and is available via a publicly accessible Azure Blob Storage Account located in the Eats-US Azure region.

In [0]:
import uuid
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Azure storage access info
blob_account_name = "azureopendatastorage"
blob_container_name = "nyctlc"
blob_relative_path = "yellow"
blob_sas_token = r""
# Allow SPARK to read from Blob remotely
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set(
  'fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name),
  blob_sas_token)
print('Remote blob path: ' + wasbs_path)
# SPARK read parquet, note that it won't load any data yet by now
df_rawInput = spark.read.parquet(wasbs_path)

# Adding an id column with unique values
uuidUdf= udf(lambda : str(uuid.uuid4()),StringType())
df_input_withId = df_rawInput.withColumn("id", uuidUdf())

print('Register the DataFrame as a SQL temporary view: source')
df_input_withId.createOrReplaceTempView('source')

**Preparation - creating the Cosmos DB container to ingest the data into**

Configure the Catalog API to be used

In [0]:
spark.conf.set("spark.sql.catalog.cosmosCatalog", "com.azure.cosmos.spark.CosmosCatalog")
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountEndpoint", cosmosEndpoint)
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountKey", cosmosMasterKey)

And execute the command to create the new container with a throughput of up-to 100,000 RU (Autoscale - so 10,000 - 100,000 RU based on scale)

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS cosmosCatalog.SampleDatabase;

CREATE TABLE IF NOT EXISTS cosmosCatalog.SampleDatabase.YellowCabSource
USING cosmos.items
TBLPROPERTIES(partitionKeyPath = '/id', autoScaleMaxThroughput = '100000');

** Sample - ingesting the NY Yellow Cab data into Cosmos DB**

In [0]:
writeCfg = {
  "spark.cosmos.accountEndpoint": cosmosEndpoint,
  "spark.cosmos.accountKey": cosmosMasterKey,
  "spark.cosmos.database": "SampleDatabase",
  "spark.cosmos.container": "YellowCabSource",
  "spark.cosmos.write.strategy": "ItemOverwrite",
  "spark.cosmos.write.bulkEnabled": "False"
}

df_NYYellowCab_Input = spark.sql('SELECT * FROM source')

df_NYYellowCab_Input \
  .write \
  .format("cosmos.items") \
  .mode("Append") \
  .options(**writeCfg) \
  .save()

**Cleanup - deleting the Cosmos DB container and database again (to reduce cost) - skip this step if you want to keep them**

In [0]:
%sql
DROP TABLE IF EXISTS SampleDatabase.YellowCabSource;

DROP DATABASE IF EXISTS cosmosCatalog.SampleDatabase CASCADE;